Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [204]:
import pandas as pd
import numpy as np
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, cross_val_score
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import torch
import transformers as ppb
from tqdm import notebook
import csv

In [134]:
sys.setrecursionlimit(7500)

In [135]:
comm = pd.read_csv('/Users/yakovlev/Downloads/toxic_comments.csv')
comm.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [136]:
comm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


В данных представлено 159571 комментариев, которые класссифицированы токсичные ли они или нет. Toxic - целевой признак, который нужно будет предсказывать. Отделим целевой признак и обычный.

In [137]:
X = comm['text']
y = comm['toxic']

Разделим данные на обучающуюю валидационную и тестовую выборки

In [138]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=2, stratify=y_train)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(95742,) (95742,)
(31914,) (31914,)
(31915,) (31915,)


Напишем функцию, которая получая предложение будет производить стемминг для каждого слова, а также избавляться от знаков препинания

In [139]:
porter = PorterStemmer()
def stemming(sentence):
    word_list = []
    for i in sentence.split():
        word_list.append(porter.stem(i))
    word_list = " ".join(word_list)
    return " ".join(re.sub(r'[^A-Za-z]', " ", word_list).split())

Посмотрим на примере, что получится после лемматизации

In [140]:
a = "I will never, ever, : play computer games"

In [141]:
stemming(a)

'I will never ever play comput game'

Применим данную функцию ко всем трем датасетам

In [ ]:
X_train = X_train.apply(stemming)
X_train.head()

In [143]:
X_val = X_val.apply(stemming)
X_val.head()

17334     lol I think that there are too mani lol in the...
60468     I agre hockeyfights com is not a reliabl sourc...
123619    paid kleargear editor take a bow woman you hav...
26664     wow thi is veri helpful i ll definit tri to ad...
123132    thi absurd He get to make ani accus he likes a...
Name: text, dtype: object

In [144]:
X_test = X_test.apply(stemming)
X_test.head()

127835    can you check on someth weird you delet thi pa...
125544    csd I just csd d robert F brands and I saw you...
10000     Hi redros and apolog for delay here is a draft...
82718     A belat welcome here wish you a belat welcom t...
3093      unrel comment a coupl of questions and advic s...
Name: text, dtype: object

### Вывод

Подготовили датасеты для перевода в вектора и обучения модели

# 2. Обучение модели

Векторизируем комментарии, но для начала выделим stopwords - слова, которые не несут смысловой нагрузки

In [145]:
stopwords = set(stopwords.words('english'))

Разобьем каждый комментарий на двухграммы и преобразуем датасет для обучения

In [146]:
tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf.fit(X_train.values.astype('U'))

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

Переведем данные в формат Unicode и трансформируем датасеты

In [147]:
X_train = tf_idf.transform(X_train.values.astype('U'))

In [148]:
X_val = tf_idf.transform(X_val.values.astype('U'))
X_test = tf_idf.transform(X_test.values.astype('U'))

Обучим модель логистической регресси

In [149]:
model = LogisticRegression(class_weight='balanced')

In [150]:
cross_val_score(model, X_train, y_train, scoring='f1', verbose=0).mean()

0.7462782369210601

Посмотрим, какие будут результаты на валидационных данных, которые мы отделили от всего датасета

In [151]:
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [152]:
f1_score(model.predict(X_val), y_val)

0.75

Посмотрим результат на тестовых данных

In [153]:
f1_score(model.predict(X_test), y_test)

0.7486427795874049

Попробуем объединить тренировочную и валидационную выборку


In [154]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [155]:
tf_idf.fit(X_train)

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [156]:
X_train = tf_idf.transform(X_train)
X_test = tf_idf.transform(X_test)

In [157]:
cross_val_score(model, X_train, y_train, scoring='f1', verbose=0).mean()

/Users/yakovlev/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/yakovlev/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

0.7500023633698109

Посмотрим на тестовых данных

In [158]:
model.fit(X_train, y_train)
f1_score(model.predict(X_test), y_test) 

0.7547735951991271

### Вывод

Мы добились нужного результата на тестовой выборке

## Обучение BERT

При обучении BERT будем использовать 100000 объектов выборки, чтобы сэкономить время

In [162]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [163]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [182]:
tokenized = X.apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))[:10000]

In [183]:
tokenized.shape

(10000,)

Выполним padding: приведем каждый токенизированный комментарий к одной длине

In [184]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [185]:
np.array(padded).shape

(10000, 357)

In [186]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 357)

Возьмем первые 10 тысяч объектов выборки

In [187]:
batch_size = 20
for i in notebook.tqdm(range(0, padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        with open('embeddings1.csv', 'a') as fp:
            for row in batch_embeddings[0][:,0,:].numpy():
                csv.writer(fp, delimiter=',').writerow(row)                

Получили новый датасет, который содержит эмбеддинги

In [188]:
embeddings = pd.read_csv('embeddings1.csv', header = None, prefix="var_")

In [203]:
embeddings.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_758,var_759,var_760,var_761,var_762,var_763,var_764,var_765,var_766,var_767
0,0.220490,-0.097744,-0.073250,-0.071392,-0.085661,-0.183036,0.322566,0.242753,-0.038898,-0.271067,...,0.177155,-0.026148,0.086816,-0.171984,0.303477,0.019842,-0.199227,0.153798,0.416238,0.403389
1,-0.118798,-0.156563,0.238368,-0.128910,-0.122927,-0.096002,0.643840,0.153690,-0.207519,-0.335488,...,0.098511,-0.217268,0.038986,-0.440684,0.285151,-0.148382,0.210516,-0.048998,0.543574,0.514805
2,0.075954,0.061317,-0.122162,-0.135580,-0.120700,-0.396361,0.039420,0.519339,-0.130743,-0.324620,...,0.017994,-0.276312,0.167717,-0.168578,0.155621,0.323910,-0.154194,0.106488,0.530345,0.335555
3,0.105888,-0.029933,0.121697,-0.183122,-0.031695,-0.380252,0.073327,0.327184,-0.139291,-0.197600,...,0.244153,-0.060619,-0.136875,-0.186269,0.243312,-0.018964,-0.356836,-0.069108,0.442633,0.405762
4,-0.116650,-0.038441,-0.080841,-0.028250,-0.014767,-0.308648,0.141258,0.533529,-0.274986,-0.334552,...,-0.012090,-0.161019,0.241565,-0.178456,0.210458,0.240435,-0.285982,0.031116,0.412562,0.304650


Выделим первые 10000 объектов в таргете

In [195]:
y = y[:10000]

In [ ]:
Поделим данные на обучающую и тестовую выборки

In [197]:
X_train, X_test = train_test_split(embeddings, test_size=0.25, shuffle=False)

In [198]:
y_train, y_test = train_test_split(y, test_size=0.25, shuffle=False)

In [199]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(7500, 768) (2500, 768)
(7500,) (2500,)


Обучим модель логистической регресси и посмотрим на значение метрики f1 на кросс-валидации

In [200]:
model = LogisticRegression(class_weight='balanced')
cross_val_score(model, X_train, y_train, scoring='f1', verbose=0).mean()

/Users/yakovlev/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/yakovlev/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

0.6700918639570131

Посмотрим, какой результат получился на тестовых данных

In [201]:
model.fit(X_train, y_train)
f1_score(model.predict(X_test), y_test)

/Users/yakovlev/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.634390651085142

### Вывод

В результате научились классифицировать комментарии на токсичность

# 3. Выводы

В результате получили, что при использовании tf-idf векторизации вместе с логистической регресией значение метрики f1 достигает 0.75, а при использовании BERT метрика показывает только 0.63 на тестовых данных. Но стоит отметить, что 0.63  было получено всего лишь на 10 тыс. объектах, а не на почти 160 тыс, как в первом случае. Стоит предположить,  что при использовании BERT на полноценной выборке мы можем получить значение метрики если не больше 0.75, то сильно близкое к этому, но для этого нужно гораздо большее количество времени

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны